# Social Graphs Project: Analyzing artists and songs

In [ ]:
#pip install spotipy
#pip install bs4
# The one below requires to have git installed
#!pip install git+https://github.com/johnwmillr/LyricsGenius.git

In [ ]:
import re
import requests
import json
import networkx as nx
from tqdm import tqdm
import json
import codecs
from requests.exceptions import ReadTimeout
import lyricsgenius

In [ ]:
CLIENT_ID = "9a8ba8428ffc4b219099ad261a1366cf"
CLIENT_SECRET = "24b64e8cfee747889094e8a09da2b8f2"

AUTH_URL = "https://accounts.spotify.com/api/token"
SPOTIFY_BASE_URL = "https://api.spotify.com/v1/"

In [ ]:

grant_type = 'client_credentials'
body_params = {'grant_type' : grant_type}

url='https://accounts.spotify.com/api/token'
response = requests.post(url, data=body_params, auth = (CLIENT_ID, CLIENT_SECRET)) 

token_raw = json.loads(response.text)
token = token_raw["access_token"]

In [ ]:
headers = {"Authorization": "Bearer {}".format(token)}
r = requests.get(url="https://api.spotify.com/v1/artists/id/0TnOYISbd1XYRBk9myaseg", headers=headers)
print(r)

# Data collection

To start with, we want a list of artists (we chose the Wikipedia list of hip-hop artist).
Once we have it, we will, for every artist, find the list of all their songs, and all of the other artists they featured on a song.

In [ ]:
'''
# ------------- Getting a list of hip-hop artists from Wikipedia -----------------------------------------------------
# We will use the names for getting info from the Spotify API
# The wikipedia links can be useful should we need to get info from the artists' pages

# artists = {artist_name: [wiki_link, ...], ...}
artists = {}

# Parsing html content with beautifulsoup
soup = BeautifulSoup(requests.get("https://en.wikipedia.org/wiki/List_of_hip_hop_musicians").content, 'html.parser')
lists = soup.find_all(["div"], {"class": "div-col"})

for l in tqdm(lists):
    for match in re.findall(r"href=\"([\w\s=\/\(\).'%+!éè%$-]*)[\w\s=\"'\/\(\).\%+!éè$-]+>([.\w\s+'!éè$-]+)", str(l)):
        artists[match[1].lower()] = []
        artists[match[1].lower()].append(match[0])
        
print(len(artists), "artists were found on the Wikipedia page and added to the dictionary.")


# ------------- Getting the Spotify URI of every artist ---------------------------------------------------

temp = len(artists)
to_remove = []

for a in tqdm(artists):
    results = sp.search(q='artist:' + a, type='artist')
    # The item number 0 is the most popular result, so it is ikely to be the artist we're looking for
    try:
        artists[a].append(results['artists']['items'][0]['uri'])
    except:
        to_remove.append(a)

for a in to_remove:
    artists.pop(a)
    
print(temp-len(artists), "artists were not found with the Spotify API and were removed from the dictionary.")

'''

In [ ]:
# Saving the dictionary to a local text file

with codecs.open("C:/Users/difto/Desktop/artists.txt", 'w', "utf-8") as f:
    f.write(json.dumps(artists))

In [ ]:
# Reloading the data from the file to make sure the cell runs on a clean version of it
with codecs.open("./artists.txt", 'r', "utf-8") as f:
    data = json.load(f)

#### From Spotify

In [ ]:
# If the cell above runs correctly, we end up with this format:
# data = {artist: [wikipedia_link, spotify_uri, [featured_artist, ...], [track_uri, ...]], ...}

In [ ]:
# Retrying all the failed requests

path = "C:/Users/difto/Desktop/failed requests.txt"

with codecs.open(path, 'r', "utf-8") as f:
        for j in re.findall(r'\d+', f.read()):
            j = int(j)
            print(j)
            
            feats = set()
            songs = set()
            try:
                response = requests.get(SPOTIFY_BASE_URL+"artists/"+data[artist_names[j]][1]+"/albums?limit=50", headers=SPOTIFY_HEADER)
                response = json.loads(response.content)
                for album in response['items']:
                    # We fetch the album data
                    response2 = requests.get(SPOTIFY_BASE_URL+"albums/"+album["id"]+"/tracks", headers=SPOTIFY_HEADER)
                    response2 = json.loads(response2.content)
                    # We go through every song on the album
                    for track_number in range(len(response2['items'])):
                        # Getting the song's name
                        songs.add(response2["items"][track_number]["name"])
                        # Getting the artists featured
                        for item in response2['items'][track_number]['artists']:
                            feats.add(item['name'].lower())
                data[artist_names[j]].append(list(feats))
                data[artist_names[j]].append(list(songs))
            except (ReadTimeout, KeyError):
                print("Did not work for", j, ":", artist_names[j])
                failedRequestIndexes.append(j)

with codecs.open("C:/Users/difto/Desktop/Cours/02805 Social graphs and interactions/Project/Source/socialgraph-final-project/Data/data_final.txt", 'w', "utf-8") as f:
        f.write(json.dumps(data))

In [2]:
# Loading the final data set

# Format:
# data = {artist: [wikipedia_link, spotify_uri, [featured_artist, ...], [song_name, ...]], ...}

with codecs.open("C:/Users/difto/Desktop/Cours/02805 Social graphs and interactions/Project/Source/socialgraph-final-project/Data/data_final.txt", 'r', "utf-8") as f:
    data = json.load(f)

#### From Genius

Now we get the lyrics from a few songs of every artist, in order to later try to identify patterns.

In [112]:
def processLyrics(text):
    # Removing all the [Chorus], [Verse 1], etc
    for a in re.findall(r'\[[\w\s:]+\]', text):
        text = text.replace(a, ' ')
    # Removing all the line breaks
    for a in re.findall(r'\n', text):
        text = text.replace(a, ' ')
    # Removing all the punctuations
    for a in re.findall(r"['\s]|(\W)", text):
        text = text.replace(a, '')
    return text.lower()

In [93]:
# Sometimes the Lyricsgenius package gives us a list of songs ("Artist - Song") instead of lyrics
# We spot this error by counting the number of dash in the text ("-")

# Some other time, the package doesn't find the actual song and returns something completely differen
# We spot this error by checking the title
def isResponseValid(artist, song_title, response_song_title, response_lyrics): 
    a = [c for c in response_song_title.lower() if c.isalnum()]
    b = [c for c in song_title.lower() + " by " + artist.lower() if c.isalnum()]
    condition1 = a[:len(b)] == b
    if not condition1:
        return False
    condition2 = (len(re.findall(r'-', response_lyrics))+len(re.findall(r'—', response_lyrics)))/len(response_lyrics)<0.01
    return condition1 and condition2

In [113]:
# Getting the lyrics of a given song from Genius

#GENIUS_CLIENT_ID = "gCDSyLMANvERLDi4GdVvKN6l7-QB1y2NHSkPr6QCUPVjKzEG_93CxN6Plj82nzWh"
GENIUS_ACCESS_TOKEN = "eAW6zYgc_Mf2dYp3xkouP8c8SuGjWg76mlVC95htgmJtOkD_OpKbwXK8aqfQMFRA"

genius = lyricsgenius.Genius(GENIUS_ACCESS_TOKEN)
genius.verbose = False
genius.remove_section_headers = True
genius.skip_non_songs = True

lyrics = {}

MIN_LYRICS_LENGTH = 10000
# We can try to get lyrics this number of times, if it doesn't work, we stop and go to the next artist
MAX_ATTEMPTS = 20

for artist in tqdm(list(data)[:1]):
    if len(data[artist])>2:
        text = ""
        song_number = 0
        while len(text) < MIN_LYRICS_LENGTH and song_number < len(data[artist][3]) and song_number < MAX_ATTEMPTS:
            song = data[artist][3][song_number]
            try:
                response = genius.search_song(song, artist)
                if isResponseValid(artist, song, response.full_title, response.lyrics):
                    # Processing the lyrics to remove unwanted notation and words
                    text += processLyrics(response.lyrics)
                song_number += 1
            except:
                song_number += 1
        lyrics[artist] = text.split()


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.82s/it]


In [114]:
lyrics['03 greedo']

['look',
 'at',
 'me',
 'now',
 'lyrics',
 'ahhah',
 'you',
 'know',
 'i',
 'always',
 'wanted',
 'to',
 'be',
 'a',
 'six',
 'figure',
 'nigga',
 'i',
 'signed',
 'a',
 'deal',
 'and',
 'got',
 'seven',
 'bitch',
 'ahhahhahhahhah',
 'ron',
 'ron',
 'do',
 'that',
 'shit',
 'look',
 'at',
 'me',
 'now',
 'bitch',
 'look',
 'at',
 'me',
 'now',
 'bitch',
 'ayy',
 'ayy',
 'you',
 'know',
 "i'm",
 "stylin'",
 'you',
 'know',
 "i'm",
 "stylin'",
 'ayy',
 'ayy',
 'castaway',
 'got',
 'away',
 'made',
 'it',
 'out',
 'the',
 'island',
 'ayy',
 'ayy',
 'mafia',
 'mafia',
 'like',
 'a',
 'damn',
 'italian',
 'ayy',
 'ayy',
 'look',
 'at',
 'me',
 'now',
 'bitch',
 'look',
 'at',
 'me',
 'now',
 'bitch',
 'ayy',
 'ayy',
 'you',
 'know',
 "i'm",
 "stylin'",
 'you',
 'know',
 "i'm",
 "stylin'",
 'ayy',
 'ayy',
 'look',
 'at',
 'me',
 'look',
 'at',
 'me',
 'know',
 'look',
 'at',
 'me',
 'look',
 'at',
 'me',
 'know',
 'look',
 'at',
 'me',
 'look',
 'at',
 'me',
 'know',
 'look',
 'at',
 'me',
 

In [118]:
import nltk
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

# Removing the stop words from the lyrics
text = [w for w in lyrics['03 greedo'] if w not in STOPWORDS]

# Creating the frequency distribution
fdist = nltk.FreqDist(text)

# Getting the most common words in the distribution
fdist.most_common()

[('ayy', 62),
 ('look', 32),
 ('bitch', 29),
 ('know', 25),
 ('yeah', 22),
 ('nigga', 20),
 ('ron', 18),
 ('shit', 17),
 ('went', 17),
 ('got', 15),
 ('niggas', 14),
 ('fuck', 14),
 ('baby', 12),
 ('tired', 12),
 ('dog', 12),
 ('now', 11),
 ('might', 11),
 ('gang', 10),
 ('beat', 9),
 ('pussy', 9),
 ('hundred', 9),
 ('hit', 8),
 ('boy', 8),
 ("ain't", 7),
 ("i'ma", 7),
 ('make', 7),
 ('drummer', 7),
 ('every', 7),
 ('03', 7),
 ('beethoven', 7),
 ('zaytoven', 7),
 ('metro', 7),
 ('boomin', 7),
 ('lyrics', 6),
 ("stylin'", 6),
 ('see', 6),
 ('told', 6),
 ('ride', 6),
 ('time', 6),
 ('go', 6),
 ('quiet', 6),
 ("takin'", 6),
 ('never', 5),
 ('real', 5),
 ('take', 5),
 ('feelings', 5),
 ('hold', 5),
 ('wanna', 5),
 ('four', 5),
 ('hood', 5),
 ('need', 5),
 ('oh', 5),
 ('ocean', 5),
 ('mafia', 4),
 ('bitches', 4),
 ("goin'", 4),
 ('whoa', 4),
 ('long', 4),
 ('bang', 4),
 ('feet', 4),
 ("touchin'", 4),
 ('ceiling', 4),
 ('close', 4),
 ('think', 4),
 ("ridin'", 4),
 ('peoples', 4),
 ('feel', 4

In [73]:
genius = lyricsgenius.Genius(GENIUS_ACCESS_TOKEN)
genius.verbose = False
genius.remove_section_headers = True
genius.skip_non_songs = True

song = "Stop Me Now"
artist = "2 chainz"

response = genius.search_song(song, artist)

a = [c for c in response.full_title.lower() if c.isalnum()]
b = [c for c in song.lower() + " by " + artist.lower() if c.isalnum()]

print(a)
print(b)
print(a[:len(b)] == b)

['s', 't', 'o', 'p', 'm', 'e', 'n', 'o', 'w', 'b', 'y', '2', 'c', 'h', 'a', 'i', 'n', 'z', 'f', 't', 'd', 'o', 'l', 'l', 'a', 'b', 'o', 'y']
['s', 't', 'o', 'p', 'm', 'e', 'n', 'o', 'w', 'b', 'y', '2', 'c', 'h', 'a', 'i', 'n', 'z']
True


In [76]:
lyrics

{'03 greedo': '',
 '070 shake': '',
 '1.cuz': '',
 '100 kila': '',
 '12 gauge': '',
 "12 o'clock": '',
 '2 black 2 strong': '',
 '2 chainz': '',
 '2 pistols': '',
 '20syl': ''}

In [52]:

genius = lyricsgenius.Genius(GENIUS_ACCESS_TOKEN)
song = "Eyes Closed"
artist = "2 pistols"
response = genius.search_song(song, artist)
print(response.full_title)
print(song + " by " + artist)

Searching for "Eyes Closed" by 2 pistols...
Done.
Eyes Closed by 2 Pistols
Eyes Closed by 2 pistols


In [21]:
text = 'uum Project (2018) Sudan Archives — Sink (2018) NDAI — "Time Heals All Wounds" (2018) Maggie Rogers — "Fallingwater" (2018) Naji — Act I (2018)June 3Silk — Silk (1995)June 2N/AJune 1Young Nudy (feat. 21 Savage) — "Since When" (2018) D.O.D — "Glow" (2018) The New Respects — "Freedom" (2018) Clairo — diary 001 (2018) Jorja Smith — "February 3rd" (2018) Juice WRLD — Goodbye & Good Riddance (2018)MayMay 31Pusha T — DAYTONA (2018) CHVRCHES — Love Is Dead (2018) Shawn Mendes — Shawn Mendes (2018) Drake — "I\'m Upset" (2018)May 30The Internet — "Roll! (Burbank Funk) (KAYTRANADA Remix)" (2018) Blue October — "Colors Collide" (2018) nobigdyl. — "twenty eighteen" (2018) nobigdyl. — "psycho heart!" (2018) Jay Rock — "WIN" (2018) Chaos Chaos — Chaos Chaos (2018) Temi Omi (feat. Tierra Whack) — "Pull Up on You" (2018) Alex Da Kid (feat. H.E.R. & Rapsody) — "Go" (2018) KYLE — Light of Mine (2018) Christina Aguilera (feat. Demi Lovato) — "Fall in Line" (2018) Lil Baby — Harder Than Ever (2018) Pharrell Williams & Camila Cabello — "Sangria Wine" (2018)May 29NIKI — "I Like U" (2018) NIKI — "See U Never" (2018) NIKI — "Chilly" (2018) NIKI — "Vintage" (2018) ORKID — "So Rude" (2018) TOMI — "Coming Around" (2018) Natalie Prass — "Lost" (2018) Now, Now — "MJ" (2018) Jennifer Hudson — "I\'ll Fight" (2018) Ashe — "Choirs" (2018) THEY. — "Ain\'t the Same" (2018) Sabrina Carpenter & Jonas Blue — "Alien (M-22 Remix)" (2018) Terrance Escobar (feat. Thouxanbanfauni) — "*D 4 L*" (2018) Mike Jones — "All on Me" (2018) A$AP Ferg & Elle Fanning — "Moon River" (2018) Jasmine Thompson — "Lonely Together" (2018) Houndmouth — "This Party" (2018) Ginny Owens — "Remind Me" (2018) Jean Deaux — "Energy" (2018) morgxn — "me without you" (2018) Gallant — "Doesn\'t Matter" (2018) Chantel Jeffries (feat. Offset & Vory) — "Wait" (2018) Tee Grizzley (feat. Chris Brown) — "Fuck It Off" (2018) Wiz Khalifa (feat. Gucci Mane) — "Real Rich" (2018) Jessie Reyez — "Body Count" (2018) Troye Sivan — "Bloom" (2018) Julia Michaels (feat. Trippie Redd) — "Jump" (2018) Jess Glynne — "I\'ll Be There" (2018) Dan + Shay — "All to Myself" (2018) Charlie Puth (feat. Kehlani) — "Done for Me" (2018) Charlie Puth — "The Way I Am" (2018) Florence + The Machine — "Hunger" (2018) Travis Scott (feat. Lil Uzi Vert & Kanye West) — "Watch" (2018) Shawn Mendes (feat. Khalid) — "Youth" (2018)May 28N/AMay 27N/AMay 26A$AP Rocky — TESTING (2018)May 25N/AMay 24N/AMay 23N/AMay 22N/AMay 21N/AMay 20N/AMay 19Various Artists — Boomerang Soundtrack (1992) Foggieraw — "Prolly" (2018) BRYN (feat. Yung Fume) — "Pull Up" (2018)May 18Desiigner — L.O.D. (2018)May 17N/AMay 16N/AMay 15N/AMay 14N/AMay 13N/AMay 12April + Vista — "Own2" (2018) Amy Grant — "Say It With a Kiss" (2018) Jodeci — The Show, the After-Party, the Hotel (1995)May 11Playboi Carti — Die Lit (2018)May 10Leon Bridges — Good Thing (2018) Rob $tone — Young Rob $tone (2018) Jeremy Zucker — glisten (2018) Yung Gravy — Snow Cougar (2018) Amber Mark — Conexão (2018)May 9BlocBoy JB — SIMI (2018) 6 Dogs & Danny Wolf — 6 Wolves (2018) Slim Jxmmi — Jxmtro (2018) Swae Lee — Swaecation (2018) Rae Sremmurd — SR3MM (2018) Tove Styrke — Sway (2018)May 8Playboi Carti — "Foreign" (2018) L.A. Salami — The City of Bootmakers (2018) Laura Veirs — The Lookout (2018) Jim Jones — Wasted Talent (2018) nothing,nowhere. — ruiner (2018)May 7Pete Rock & C.L. Smooth — The Main Ingredient (1994) Childish Gambino — "This Is America" (2018) Madeline Kenney — Night Night at the First Landing (2017) Annmarie — Tripolar (2018) Georgi Jacobs — "Hard to Love" (2018) Elle Teresa — KAWAII BUBBLY LOVELY (2018)May 6702 — No Doubt (1996) A$AP Ferg — Still Striving (2017) Madeline Kenney — Signals (2016) 21 Savage & Metro Boomin — Savage Mode (2016) Trippie Redd — A Love Letter to You (2017) Wintertime — I Know What You Did Last Winter (2016) Cassy London — Realm (2017)May 5Cassy London — "Wonderland" (2016) Diego Money — Go Diego EP (2017)May 4Diego Money — Diego & Friends (2017) Evangeline Joy — Verbatim (2013) Karen Waldrup — "Warm in Your Sunshine" (2017) Phoebe Ryan — James (2017) Nadia — Rebel (2017) Jxck Kelly — "Nancy" (2018) Jxck Kelly — "Nothing" (2018) K. Cero — Songs About You (2017) Soduh — The Crocodile (2018)May 3Phoebe Ryan — Mine EP (2015) Ashaya — November (2016) Evan Diamond — "Intersections" (2018) Vai Petri — "Criminal" (2018) Vai Petri — "Let Live" (2017) Analisa Corral — Sugar Pills - Pack 1 (2018) Kara Elyse — "Take Control" (2018) Aced Spade (feat. Lé, Kiim Bosh, D. Laflair & Marko Stat$) — "Outta Sight" (2018)May 2Carnage — Battered Bruised & Bloody (2018) Ciaran Lavery — Sweet Decay (2018) Robert Glasper Experiment — Robert Glasper x KAYTRANADA: The ArtScience Remixes (2018) Robotaki — Science (2018) Skizzy Mars — are you OK? (2018) Bazzi — COSMIC (2018)May 1Famous Dex — Dex Meets Dexter (2018) The Aces — When My Heart Felt Volcanic (2018) The Longshot — The Longshot EP (2018) Luniz — No Pressure (2018) Ski Mask The Slump God — Beware the Book of Eli (2018) Kanye West — "Lift Yourself" (2018) Cardi B — Invasion of Privacy (2018) Serayah — Addicted (2018) Sigrid — Raw (2018) Smokepurpp & Murda Beatz — Bless Yo Trap (2018) Young Thug — Hear No Evil (2018) isaac gracie — isaac gracie (2018) Sofi Tukker — Treehouse (2018) Alyson Stoner — "God\'s Plan" (2018) Moon Taxi (feat. Sofi Tukker) — "Two High (Sofi Tukker Remix)" (2018) Terri Clark & Dallas Smith — "One Drink Ago" (2018) Ne-Yo, Bebe Rexha & Stefflon Don — "PUSH BACK" (2018) Daryl Hall & John Oates (feat. Train) — "Philly Forget Me Not" (2018) THEY. & DEAN — "Dante\'s Creek (deantrbl Remix)" (2018) Nelly (feat. Jacquees) — "Freaky with You" (2018) Amine (feat. Injury Reserve) — "Campfire" (2018) Ralo & P.Skud — "Don\'t Worry" (2018) Bebe Rexha — "Meant to Be (Acoustic)" (2018) Jenny Hval — "Spells" (2018) Calpurnia — "Louie" (2018) DREAMS — "Silence" (2018) Gang Gang Dance — "Lotus (edit)" (2018) Royce da 5\'9" (feat. Ashley Sorrell) — "Stay Woke" (2018)AprilApril 30Chaos Chaos — "Pink Politics" (2018) La Luz — "The Creature" (2018) REMMI — "Desperate" (2018) Couros — "Young" (2018) Highasakite — "Out of Order" (2018) Mavi Phoenix — "Yellow" (2018) TOMI — "Every Morning I Feel Like Running Away" (2018) Wet — "Soften" (2018) Asteph — "No One Will Smile" (2018) AUGUST 08 — "Lately" (2018) Alae — "Stone Cold" (2018) Jackson Penn — "Babylon" (2018) Morgan Wallen — "Happy Hour" (2018) Allen Stone — "Brown Eyed Lover" (2018) Higher Brothers & DJ Snake — "Made in China" (2018) The Shires — "The Hard Way" (2018) Dierks Bentley — "The Mountain" (2018) Kideko — "Good Thing" (2018) Marian Hill — "Wish You Would" (2018) Powers Pleasant (feat. Joey Bada$$ & A$AP Ferg) — "Pull Up" (2018) Tritonal (feat. Riley Clemmons) — "Out My Mind (Club Mix)" (2018) Bearson (feat. Ashe) — "Get Lost" (2018) Cole Swindell  — "Somebody\'s Been Drinkin\'" (2018) Ekali & Medasin (feat. Elohim) — "Forever" (2018) Christopher — "Bad" (2018) Amy Shark — "I Said Hi" (2018) 50 Cent (feat. PnB Rock) — "Crazy" (2018) SATICA — "Inner Child" (2018) DJ Megan Ryte (feat. Tory Lanez & HoodCelebrityy) — "On & On" (2018) John Splithoff — "Raye" (2018) Gavin James — "Always" (2018) Aries — "SAYONARA" (2018) Juice WRLD — "All Girls Are the Same" (2018) Brandy Burnette & Dwilly (feat. Kayo Genesis) — "Hands Down" (2018) BATE (feat. Blake Rose & Radio 3000) — "One I Love" (2018) Florence + the Machine — "Sky Full of Song" (2018) Cash Cash (feat. Abir) — "Finest Hour" (2018) Louis the Child (feat. Wafia) — "Better Not" (2018) 5 Seconds of Summer — "Youngblood" (2018) ZAYN — "Let Me" (2018) G-Eazy (feat. Yo Gotti & YBN Nahmir) — "1942" (2018) Gianna Adams — I Don\'t Know (2018) Drake — "Nice for What" (2018) Catherine McGrath — "Wild" (2018) Trinidad Cordova — "Even If" (2018) Youngr — "Drive" (2018) Marshmello & Anne-Marie (feat. A Boogie Wit da Hoodie) — "FRIENDS (A Boogie Wit da Hoodie Remix)" (2018)April 29Empress Of — Trust Me Baby / In Dreams (2018) Derek Minor (feat. Dre Murray, Chino Dollaz & Anesha Birchett) — "Decisions" (2018) Phora — "Come Thru" (2018) nobigdyl. — "enemies?" (2018) Patrick Paige II (feat. Syd & Kari Faux) — "On My Mind / Charge It to the Game" (2018) Nicki Minaj — "Chun-Li" (2018) King Princess — "Talia" (2018) Swae Lee (feat. Slim Jxmmi) — "Guatemala" (2018) Lil Pump — "Esskeetit" (2018) Bebe Rexha (feat. Quavo) — "2 Souls on Fire" (2018) Bebe Rexha — "Ferrari" (2018) Nicki Minaj — "Barbie Tingz" (2018) Keith Urban — Graffiti U (2018) Half Waif — Lavender (2018) Janelle Monáe — Dirty Computer (2018) We Are Scientists — Megaplex (2018) Cicada Rhythm — Everywhere I Go (2018) Twin Shadow — Caer (2018)April 28Elohim — Elohim (2018) Sarah Reeves — Easy Never Needed You (2018) Buzzy Lee — Facepaint (2018) XXXTENTACION — ? (2018) Anne-Marie — Speak Your Mind (2018) SahBabii (feat. 21 Savage) — "Outstanding" (2018) Peso da Mafia — "About Us" (2018) Codes — GO DOWN / THAT CRACK (2018)April 27NEEDTOBREATHE — "Darling" (2018) Kailee Morgue — "Fuck U" (2018) Meg Myers — "Numb" (2018) Shea Diamond — "Keisha Complexion" (2018) A Boogie Wit da Hoodie — "Right Moves" (2018) Tee Grizzley (feat. Lil Yachty) — "2 Vaults" (2018) Aloe Blacc — "Brooklyn in the Summer" (2018) The Internet — "Roll! (Burbank Funk)" (2018) Clairo — "4EVER" (2018) Jason Mraz — "Have It All" (2018) Imagine Dragons — Evolve (2017) Onyx — Black Rock (2018) Young Scooter — Trippple Cross (2018)April 26Towkio — WWW. (2018) Black Milk — FEVER (2018) Blac Youngsta — 223 (2018) Quinn XCII — The Story of Us (Deluxe Tracks) (2018) Quinn XCII — The Story of Us (2017)April 25J. Cole — "Want You to Fly" (2017) Skool Boy (Skoolie 300) — After Skool Vol. 1 (2009)April 24S. Carey — Hundred Acres (2018) SOB X RBE — GANGIN (2018) Kodak Black — Heart Break Kodak (HBK) (2018) Rockstar Marqo — Fuck Rap\'n (2018) The Dopplegangaz — "Fajita Effect" (2018) Ugly God — just a lil something before the album... (2018)April 23E-40 & B-Legit — Connected and Respected (2018) ALLBLACK — Outcalls (2018) Katelyn Tarver — "LY4L" (2018) Jade Bird — "Cathedral (Acoustic)" (2017) Princess Nokia — A Girl Cried Red (2018) Tinashe — Joyride (2018) The Paper Kites — On the Train Ride Home (2018) Mýa — T.K.O. (The Knock Out) (2018) Bishop Briggs — Church of Scars (2018) Lord Huron — Vide Noir (2018) The Chainsmokers — Sick Boy (2018) Sasha Sloan — sad girl (2018) Anna Leone — Wandered Away (2018) Carrie Underwood — "Cry Pretty" (2018) Lykke Li — deep end / hard rain (2018) PRETTYMUCH — PRETTYMUCH an EP (2018)April 22A$AP TyY — Best Kept Secret (2016) A$AP TyY — Troubles of the World (2018) A$AP Rocky — "Herojuana Blunts" (2018) Thouxanbanfauni — The EXFiles (2017) XVL Hendrix — Blessings & a Curse 2 (2018) Thouxanbanfauni — Heavy Weight Champ (2017) Flatbush ZOMBiES — Vacation in Hell (2018)April 21Migo Domingo — Menace II Society (2018) A$AP Ant — The Best of A$AP Ant 2017 (2018) Off Bloom — "Love Actually" (2018) Halsey (feat. Stefflon Don) — "Alone (Calvin Harris Remix)" (2018) Jack & Jack — "Stay with Me" (2018) JGrrey — "Growing" (2018) FIDLAR — "Alcohol" (2018) Mahalia — "No Reply" (2018) AURORA — "Queendom" (2018) Channel Tres — "Controller" (2018) Hobo Johnson — "Peach Scone" (2018) Calvin Harris & Dua Lipa — "One Kiss" (2018) Wiz Khalifa — "420 Freestyle" (2018) Schilly (feat. Nessly) — "Higher" (2018) Caitie Hurst — "How Could I Be Silent" (2018) Vacationer — "Being Here" (2018) KB — "No Chains" (2018) Cam — "Palace (Cover)" (2018) L.I.F.T — "WANNA DIE" (2018) Anik Khan (feat. Burna Boy & Sango) — "Oh My" (2018) Naughty Boy, RAY BLK & Wyclef Jean — "All or Nothing" (2018) Hyphen Hyphen — "KND" (2018) Rae Sremmurd (feat. Travis Scott) — "CLOSE" (2018) Jeremy Zucker (feat. Chelsea Cutler) — "better off" (2018) 6 Dogs (feat. Lil Skies) — "Off the Gas" (2018) Jean Deaux — "SSN (They Say)" (2018) CRUISR — "Mind Eraser" (2018) Anne-Marie — "2002" (2018) Caitie Hurst — "Nothing to Hide" (2018) Leon Bridges — "Beyond" (2018) iZNiiK — "round the world" (2018) BURNS (feat. Maluma & Rae Sremmurd) — "Hands on Me" (2018) A Boogie Wit da Hoodie (feat. Tory Lanez) — "Best Friend" (2018) Khalid, 6LACK & Ty Dolla $ign — "OTW" (2018) Prince — "Nothing Compares 2 U" (2018) VHS Collection — "One" (2018) Courtney Barnett — "City Looks Pretty" (2018) VanJess — "Addicted" (2018) Casey Veggies (feat. Wiz Khalifa) — "Show Off" (2018) Bas (feat. Correy C) — "Pinball II" (2018) Shae Jacobs — "Space" (2018) Confetti — "When I Grow Up" (2018) Bahari — "Savage" (2018) Sauce Walka — "No Recess" (2018) Warhol.SS — "Birds Nest" (2018)April 20Billie Eilish & Khalid — "lovely" (2018) Russ (feat. Jessie Reyez) — "Basement" (2018) Janelle Monáe — "I Like That" (2018) Ariana Grande — "no tears left to cry" (2018) J. Cole — KOD (2018)April 19Yung Bans — Yung Bans Vol. 4 (2018) Berhana — "Whole Wide World" (2018) RuthAnne — "The Vow" (2018) Ray LaMontagne — "Such a Simple Thing" (2018) Sarah Reeves — "Something About You" (2018) BlocBoy JB (feat. 21 Savage) — "Rover 2.0" (2018) San Holo (feat. Taska Black) — "Right Here, Right Now" (2018) Underoath — "Rapture" (2018) Snail Mail — "Pristine" (2018) Mr. Popular — "Price Is Right" (2018) Marc Scibilia — "Over You" (2018) Eminem (feat. Kehlani) — "Nowhere Fast (Extended Version)" (2018) Steve Angello & Sam Martin — "Nothing Scares Me Anymore" (2018) Thandi Phoenix — "My Way" (2018) Steve Aoki & Quintino — "Mayhem" (2018) CVIRO & GXNXVS — "Lady" (2018) Chloe x Halle — The Kids Are Alright (2018) Alec King — "Keep It Real" (2018) Spencer Ludwig — "Just Wanna Dance" (2018) Goody Grace — Infinite (2018) joan — "i loved you first" (2018) Ernest K., Case Arnold & Bryant Taylorr — "I Love My City" (2018) Jack & Jack — "I Don\'t Know" (2018) Nightly — "Holding On" (2018) Bazzi — "Honest" (2018) Hiss Golden Messenger — Hiss Golden Messenger Meets Spacebomb (2018) FINNEAS — "Heaven" (2018) No Jumper (feat. Tay-K & BlocBoy JB) — "Hard" (2018) 070 Shake — Glitter (2018) Sonta — In My Feelings (2017) Jason Aldean — "Gettin\' Warmed Up" (2018) NONONO — "Friends" (2018) kiLL edward — "Tidal Wave (just a little reference)" (2018) Two Door Cinema Club — Four Words to Stand On (2018) Kodaline — "Follow Your Fire" (2018) David Guetta & Sia — "Flames" (2018) Golden Features — "Falling Out" (2018) Let\'s Eat Grandma — "Falling into Me" (2018) Mishegas — "Episodes" (2018) Grizfolk — "Endless Summer" (2018)April 18Trouble & Mike WiLL Made-It — Edgewood (2018) CHVRCHES — "Miracle" (2018) Ski Mask The Slump God — "SKIMeetsWorld" (2018) ELEL — "Early in the Morning" (2018) Snow Patrol — "Don\'t Give In" (2018) Sheck Wes — "Do That" (2018) Marian Hill — "Differently" (2018) Ryan Hurd — "Diamonds or Twine" (2018) Autograf (feat. Lils & Bonsai Mammal) — "Dead Soon" (2018) Lil Skies (feat. Rich The Kid) — "Creeping" (2018) Burak Yeter — "Crash" (2018) Keith Urban (feat. Julia Michaels) — "Coming Home" (2018) Diplo — California (2018) NEEDTOBREATHE — "Bridges Burn" (2018) E^ST — "Blowjob" (2018) SHY Martin — "Bad in Common" (2018) The Shadowboxers — Apollo (2018) Vory — "9.22" (2018) The Aces — "Waiting for You" (2018) Zaytoven (feat. Yung Cakes) — "Time Too" (2018) Mike Jones — "Some Show Me Love" (2018) Pete Rock (feat. Amxxr) — "Rap Caviar" (2018) RaeLynn — "Queens Don\'t" (2018) SD — Pay Attention (2018) Noname (feat. Sniperghini) — "Nikey" (2018) The Neighbourhood — The Neighbourhood (2018) Jose Guapo — Lingo 2: The Return (2018) Chief Keef — The Leek Volume 4 (2018)April 17Lil Yachty — Lil Boat 2 (2018) Migos (feat. Hoodrich Pablo Juan) — "Migo Pablo" (2018) Foggieraw — Fogtavious Vandross (2018) Gravediggaz — "Enter the Graveyard Chamber" (2018) Cautious Clay — "Cold War (Stripped)" (2018) AJR — "Burn the House Down" (2018) Waka Flocka Flame, Zaytoven & Big Bank — The Brick House Boyz (2018) Halsey (feat. Big Sean & Stefflon Don) — "Alone" (2018) Mary Chapin Carpenter — Sometimes Just the Sky (2018)April 16Frank Ocean — Endless (2016) Her — Her (2018) Zhané — Pronounced Jah-Nay (1994) LSG — Levert.Sweat.Gill (1997) Cardi B — "Be Careful" (2018)April 15Tove Styrke (feat. Björn Yttling) — "Say My Name (Acoustic Version)" (2017) Tove Styrke — "... Baby One More Time" (2015) Tove Styrke — "liability (demo)" (2017) Tove Styrke — "Mistakes" (2017) Tove Styrke — "Changed My Mind" (2018) Tove Styrke — "Say My Name" (2017) Tove Styrke — "On the Low" (2018) OutKast — Southernplayalisticadillacmuzik (1994) Steffany Gretzinger — Blackout (2018)April 14N/AApril 13Doja Cat — Amala (2018) A Tribe Called Quest — Beats, Rhymes and Life (1996) A Tribe Called Quest — Midnight Marauders (1993) Jean Grae & Quelle Chris  — Everything\'s Fine (2018)April 12Ashley McBryde — Girl Going Nowhere (2018)April 11Craig Mack (feat. Q-Tip) — "Get Down (Q-Tip Remix)" (1994) Damar Jackson — Unfaithful (2018) The Voidz — Virtue (2018)April 10DJ Esco — KOLORBLIND (2018) Bloodboy — "Is Now a Good Time to Ruin Your Life?" (2018) Kylie Minogue — "Raining Glitter" (2018) Hinds — "Finally Floating" (2018) Andra Day — "I Wish I Knew How It Would Feel to Be Free" (2018) CHVRCHES — "Never Say Die" (2018) Tinashe (feat. Ty Dolla $ign & French Montana) — "Me So Bad" (2018) Billie Eilish — "bitches broken hearts" (2018) Grace VanderWaal — "Clearly" (2018) Tori Kelly (feat. The HamilTones) — "Help Us to Love" (2018) Lizzo — "Fitness" (2018) Cassadee Pope — "Take You Home" (2018) Jessie Reyez (feat. Daniel Caesar) — "Figures, a Reprise" (2018)April 9Henry Green — Shift (2018)April 8N/AApril 7A$AP Rocky (feat. BlocBoy JB) — "Bad Company" (2018) Ty Dolla $ign (feat. Gucci Mane & Quavo) - "Pineapple" (2018) ARTY — "Rain" (2018) Tyler, The Creator — "OKRA" (2018) Lauv — "Chasing Fire" (2018)April 6Hayley Kiyoko — Expectations (2018)April 5Rich The Kid — The World Is Yours (2018) Kacey Musgraves — Golden Hour (2018) The Weeknd — My Dear Melancholy, (2018) Louisa (feat. 2 Chainz) — "YES" (2018) Alyson Stoner — "When It\'s Right" (2018)April 4D\'Angelo — Brown Sugar (1995) Troop — Attitude (1989)April 3Christopher Williams — Changes (1992)April 2Guy — The Future (1990) Silk — Lose Control (1992)April 1DJ SpinKing (feat. A$AP Ferg, Tyga, Velous & Jeremih) — "Adult Swim" (2014) Busta Rhymes & Q-Tip — The Abstract and the Dragon (2013)MarchMarch 31H-Town — Fever for da Flavor (1993)March 30The Internet — Feel Good (2013) The Internet — Ego Death (2015)March 29N/AMarch 28N/AMarch 27N/AMarch 26N/AMarch 25N/AMarch 24Belly (feat. YG) — "4 Days" (2018) Thirty Seconds to Mars (feat. A$AP Rocky) — "One Track Mind" (2018) Kacey Musgraves — "High Horse" (2018) Shawn Mendes — "Lost in Japan" (2018) grandson — "thoughts & prayers" (2018) Panic! At the Disco — Say Amen for Silver Linings (2018) KYLE (feat. Kehlani) — "Playinwitme" (2018) Amara La Negra — "Insecure" (2018) Keith Urban (feat. Julia Michaels) — "Coming Home" (2018) Shawn Mendes — "In My Blood" (2018)March 23at Wendys — We Beefin? (2018) Tove Styrke — Kiddo (2018) Warhol.SS — Where\'s Warhol? (2017) MadeinTYO — True\'s World (2017) StaRR Lyfe — "Fuck Tomi Lahren (Shut Up Bitch)" (2016) Warhol.SS — Warhol 16\' (2017) Warhol.SS — 3200 (2017) Jeremih — The Chocolate Box (2018)March 22Playboi Carti — "Shake N Bake" (2018) Sammie — Series 31873.0 (2018) Marshmello (feat. Leah Culver) — "Fly" (2018) Steffany Gretzinger — "Save Me" (2018) Jason Derulo — "Colors" (2018) FLETCHER — "I Believe You" (2018) KYLE — "To the Moon" (2018) Vince Staples — "Get the Fuck Off My Dick" (2018) DROELOE — "Many Words" (2018) Buddy (feat. A$AP Ferg) — "Black" (2018) Phora — "Boss Up" (2018) The Knocks (feat. Foster The People) — "Ride or Die" (2018) Neko Case — "Hell-On" (2018) REMMI — "Minimum Wage" (2018) morgxn — "translucent" (2018) Ashley Monroe — "Paying Attention" (2018) The Aces — "Just Like That" (2018) Sauce Walka — "OK" (2018) Trinidad James — "T.I.B.E. (This Is Black Excellence)" (2018) Bankroll Fresh — "Whole 4" (2018) Loote — "Your Side of the Bed" (2018) JP Saxe — "The Few Things" (2018) Years & Years — "Sanctify" (2018) Alison Wonderland — "No" (2018) Wifisfuneral — "25 Lighters" (2018) Johnny Yukon — "Lonely" (2018) Yoke Lore — "Fake You" (2018) Sjowgren — "Beginners" (2018) Barns Courtney — "Sinners" (2018) Brett Castro — "Much About Sadness" (2018) Louis The Child (feat. Joey Purp) — "Shake Something" (2018) Jordan Hollywood — "Cash Out" (2018) Parquet Courts — "Wide Awake" (2018) Wet — "There\'s a Reason" (2018) Liv Dawson — "Talk" (2018) Dermot Kennedy — "Young & Free" (2018) Beach House — "Dive" (2018) Jon Hopkins — "Emerald Rush" (2018) Sofi Tukker — "Baby I\'m a Queen" (2018) TRACES — "Before I Wilt" (2018) Spazz Cardigan — "Episode" (2018) CXLOE — "Monster" (2018)March 21Necklace — "Easy" (2018) George Ezra — "Hold My Girl" (2018) Sea Girls — "Eat Me Whole" (2018) Shaylen — "Take It Back" (2018) Salt Cathedral — "No Love" (2018) Lil Jay Brown — Money Luvin Youngin: The Lick Tape (2018) John Wic — One Gunman (2018) Famous Dex & Polo Boy Shawty — When Polo Met Dexter (2018)March 20Astrid S — Party\'s Over (Acoustic) (2017) Bell Biv DeVoe — Poison (1990) Mobb Deep — The Infamous Archives (2007) Sy Ari da Kid — After the Heartbreak (2018)March 19Jonathan Wilson — Rare Birds (2018) Current Joys — A Different Age (2018)March 18Cape Cope — How to Socialise & Make Friends (2018)March 17N/AMarch 16N/AMarch 15Sabrina Carpenter & Jonas Blue — "Alien" (2018) Katie Herzig — Moment of Bliss (2018)March 14Haley Heynderickx — I Need to Start a Garden (2018)March 13The Breeders — All Nerve (2018) Waka Flocka Flame — Benjamin Flocka (2011)March 12Moby — Everything Was Beautiful, and Nothing Hurt (2018) The Undercover Dream Lovers — In Real Time (2018) Larry June — You\'re Doing Good (2018) Moss Kena — Found You in 06 (2018) JOY. — Six (2018)March 11N/AMarch 10Mikky Ekko — ADVANCE COPY EP (2018) Margaret Glaspy — Born Yesterday (2018) Valee — GOOD Job, You Found Me (2018) Soccer Mommy — Clean (2018)March 9Nessly — Wildflower (2018) ALMA — Heavy Rules Mixtape (2018) Superorganism — Superorganism (2018) Lucius — NUDES (2018) Mt. Joy — Mt. Joy (2018)March 8Tory Lanez — MEMORIES DON\'T DIE (2018)March 7B5 — B5 (2005) Digable Planets — Blowout Comb (1994)March 6Digable Planets — Reachin\' (A New Refutation of Time and Space) (1993) B2K — B2K (2002)March 5Sophie Rose — "Pink Lipstick" (2018) Subway — Good Times (1995) Katelyn Tarver — "Never Fade" (2018) Casey Veggies (feat. 03 Greedo) — "Broke" (2018) UnoTheActivist (feat. Rich The Kid) — "Fabo (Remix)" (2018) Madge — "Fight or Flight Club" (2018) Frankie Simone — "War Paint" (2018) Funky Craig & Tommie Sunshine — "Where the Drums At" (2018) BOBI ANDONOV — "Smoke" (2018) Molly Moore — "Tough Love" (2018) Maths Time Joy (feat. Flores) — "Going Nowhere" (2018) What So Not (feat. Winona Oak) — "Beautiful" (2018) Saba — "Busy" (2018) TV Noise (feat. Drew Love) — "Run" (2018) Now, Now — "AZ" (2018) Amber Mark — "Love Me Right" (2018) Northern Faces — "Firecracker" (2018) Carlos Vara — "Numb" (2018) The Neighbourhood — "Nervous" (2018) pronoun — "run" (2018) XXXTENTACION — "changes" (2018) Young Fathers — "Toy" (2018) EBEN — "LAMBO" (2018) Fickle Friends — "Wake Me Up" (2018) XXXTENTACION — "SAD!" (2018) Zaytoven (feat. Ty Dolla $ign, Jeremih & OJ da Juiceman) — "What You Think" (2018) Hoodrich Pablo Juan (feat. Gucci Mane) — "We Don\'t Luv Em (Remix)" (2018) Smooky MarGielaa — "Smooky\'s Story" (2018) Sauce Walka — "Sauce Baby" (2018) Lil Yachty (feat. Digital Nas) — "Revenge" (2018) Flatbush Zombies — "Headstone" (2018) Kap G & Hoodrich Pablo Juan — "Boss Luv" (2018) Jose Guapo (feat. Young Scooter) — "Blame It on the Streets" (2018) Miriam Bryant — "Black Car" (2018) nothing,nowhere. — "hammer" (2018) Coast Modern — "Electric Feel" (2018) Godsmack — "Bulletproof" (2018) The Moondoggies — "Sick in Bed" (2018) Hawthorne Heights — "Pink Hearts" (2018) Everette — "Slow Roll" (2018) Natalie Prass — "Short Court Style" (2018) Fedel — Gold Package (2018) The Blaze — "HEAVEN" (2018) The Kills — "Steppin\' Razor" (2018) The Kills — "List of Demands (Reparations)" (2018)March 4Little Dragon — "Best Friends" (2018) Ryan Kinder — "Leap of Faith" (2018) Smokepurpp & Murda Beatz — "123" (2018) Tom Misch (feat. De La Soul) — "It Runs Through Me" (2018) SG Lewis — "Coming Up" (2018) Rudy Mancuso & Maia Mitchell — "Magic" (2018) Jordan Davis — "Going \'Round" (2018) LA Riots — Do Your Thing/All Good (2018) Joyner Lucas & Chris Brown — "Stranger Things" (2018) Fisher — "Crowd Control" (2018) MVEK — "Tongue" (2018) Flo Rida — "Dancer" (2018) Jack White — "Over and Over and Over" (2018) Gucci Mane (feat. Migos & Lil Yachty) — "Solitaire" (2018) Meghan Trainor — "No Excuses" (2018) CHVRCHES (feat. Matt Berninger) — "My Enemy" (2018) DJ Khaled (feat. JAY Z, Future & Beyoncé) — "Top Off" (2018) Zapp — Zapp II (1982) Illegal — The Untold Truth (1993) Another Bad Creation — It Ain\'t What U Wear, It\'s How U Play It (1993) Another Bad Creation — Coolin\' at the Playground Ya Know! (1991) Zapp — Zapp (1980) Marqo 2 Fresh — Marqo da Monster (2016) N II U — N II U (1994)March 3Young Nudy (feat. Hoodrich Pablo Juan) — "Do It" (2016) RX Peso — Rixh Drugs (2018)March 2Playboi Carti — "Shawty in Love" (2018) Confetti — "People Suck" (2018) Max Frost — "Good Morning" (2018) Svrcina — "Whispers" (2018) Ramz — "Power" (2018) Jake Miller (feat. Jabbar Hakeem) — "The Girl That\'s Underneath" (2018) Sigala (feat. Paloma Faith) — "Lullaby" (2018) VanJess (feat. Berhana & Leikeli47) — "Easy" (2018) Yung Fume & Zaytoven (feat. Young Nudy) — "Something Else" (2018) Timeflies — "Are You Down" (2018) Pentatonix — "Havana" (2018) Tritonal (feat. Riley Clemmons) — "Out My Mind" (2018) Preme — "No Defeat" (2018) Ski Mask The Slump God — "DoIHaveTheSause?" (2018) Sophie Simmons — "Black Mirror" (2018) The Night Game — "Bad Girls Don\'t Cry" (2018) Clara Mae — "I Forgot" (2018) George Taylor — "Stay Warm" (2018) KLP (feat. Milwaukee Banks) — "Carried Away" (2018) Hembree — "Continents" (2018) Carlie Hanson — "Us" (2018) Vancouver Sleep Clinic — "Ayahuasca" (2018) Alex Lustig (feat. Akacia) — "In the End" (2018) Robinson — "Nothing to Regret" (2018) Nessly (feat. Joji) — "Make It Right" (2018) NoMBE (feat. Big Data) — "Drama" (2018) RAYE, Mabel & Stefflon Don — "Cigarette" (2018) Riz La Vie — "Napkins" (2018) JOHN.k — "Best of Me" (2018) Yuno — "No Going Back" (2018) Emma McGrath — "Love You Better" (2018) Superorganism — "Reflections on the Screen" (2018) King Princess — "1950" (2018) Mickey Shiloh (feat. Rami Beatz) — "Funny When" (2018) Jade Bird — "Lottery (Acoustic)" (2018) Ray J (feat. Kid Ink) — "Feeling Like Love" (2018) Foggieraw (feat. DJ Yung Vamp) — "Moncler Bubble" (2018) The Aces — "Lovin Is Bible" (2018) Noel Gallagher\'s High Flying Birds — "God Help Us All (Demo Version)" (2018) Nohidea & killedmyself — "and then, it swallowed me" (2018) Spice 1 — "2 Many Movies" (2018) Haux — "Ricochet" (2018) Ashley Monroe — "Hands on You" (2018) Zilo — "Keep Up Wimi" (2018) 24hrs (feat. Lil Pump) — "Lie Detector" (2018) Janelle Monáe — "Django Jane" (2018) GASHI — "1134" (2018) CeeLo Green — "Brick Road" (2018) Lewis Capaldi (feat. Jessie Reyez) — "Rush" (2018) Kacey Musgraves — "Space Cowboy" (2018) Bishop Briggs — "White Flag" (2018) 5 Seconds of Summer — "Want You Back" (2018) Bazzi — "Gone" (2018) David Guetta, Martin Garrix & Brooks — "Like I Do" (2018) Kacey Musgraves — "Buterflies" (2018) Janelle Monáe — "Make Me Feel" (2018) Logic — "44 More" (2018) Post Malone (feat. Ty Dolla $ign) — "Psycho" (2018) Stalley — Tell the Truth Shame the Devil, Vol. 2 (2018) Stalley — Tell the Truth Shame the Devil (2017) Cautious Clay — Blood Type (2018)March 1Holly Miranda — Mutual Horse (2018) Caroline Rose — LONER (2018) Bea Miller — aurora (2018)FebruaryFebruary 28B.o.B — B.o.B Presents: The Adventur'
isResponseValid(text)

False

# Creating the graph

Now we can create the graph, following these rules:

1. Every node corresponds to one artist
1. An edge exists between two nodes if and only if the two corresponding artists are featured on a song (made by one of the two artists, i.e. for now we don't handle the case where, say, 3 artists are on a same song)

In [42]:
# Creating the graph without node attributes for now

G = nx.Graph()

for artist in data:
    G.add_node(artist)
    if len(data[artist]) > 2:
        for featured_artist in data[artist][2]:
            if featured_artist in data:
                G.add_edge(artist, featured_artist)

angie martinez
